In [16]:
%%time

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2, os, random
import plotly
import plotly.graph_objs as go



import glob
import tensorflow
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, BatchNormalization

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split

#utility
from tqdm.notebook import tqdm
directory = 'data/chest_xray/chest_xray'

Wall time: 0 ns


In [10]:
## Check GPU

print(tensorflow.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Loading Dataset

In [24]:
train_df = glob.glob('data/chest_xray/train/**/*.jpeg')
train_df[:5]

['data/chest_xray/train\\NORMAL\\IM-0115-0001.jpeg',
 'data/chest_xray/train\\NORMAL\\IM-0117-0001.jpeg',
 'data/chest_xray/train\\NORMAL\\IM-0119-0001.jpeg',
 'data/chest_xray/train\\NORMAL\\IM-0122-0001.jpeg',
 'data/chest_xray/train\\NORMAL\\IM-0125-0001.jpeg']

In [25]:
test_df = glob.glob('data/chest_xray/test/**/*.jpeg')
test_df[:5]

['data/chest_xray/test\\NORMAL\\IM-0001-0001.jpeg',
 'data/chest_xray/test\\NORMAL\\IM-0003-0001.jpeg',
 'data/chest_xray/test\\NORMAL\\IM-0005-0001.jpeg',
 'data/chest_xray/test\\NORMAL\\IM-0006-0001.jpeg',
 'data/chest_xray/test\\NORMAL\\IM-0007-0001.jpeg']

In [26]:
validation_df = glob.glob('data/chest_xray/val/**/*.jpeg')
validation_df[:5]

['data/chest_xray/val\\NORMAL\\NORMAL2-IM-1427-0001.jpeg',
 'data/chest_xray/val\\NORMAL\\NORMAL2-IM-1430-0001.jpeg',
 'data/chest_xray/val\\NORMAL\\NORMAL2-IM-1431-0001.jpeg',
 'data/chest_xray/val\\NORMAL\\NORMAL2-IM-1436-0001.jpeg',
 'data/chest_xray/val\\NORMAL\\NORMAL2-IM-1437-0001.jpeg']

## Data Exploration

In [30]:
print("Training dataset contains {} images.".format(len(train_df)))
print("Testing dataset contains {} images.".format(len(test_df)))
print("Validation dataset contains {} images.".format(len(validation_df)))

Training dataset contains 5216 images.
Testing dataset contains 624 images.
Validation dataset contains 16 images.


## Data Visualization